In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
data_folder = 'att_hackathon/datasets'

In [6]:
#_file = 'ML_DATASET_Hackathon_Supervised.csv'
_file = 'att_hackathon/datasets/version_2.csv'

In [7]:
def load_file(path):
  df = pd.read_csv(path)
  return df

def select_cutoff(df, cutoff):
  df = df[df['percentile'] <= cutoff]
  return df

def get_non_stratifiable(df, count):
  counts = pd.DataFrame(df.Team.value_counts()).reset_index()
  non_stratifiable = list(counts[counts['Team'] < count]['index'])
  data_to_strat = df[~df.Team.isin(non_stratifiable)]
  extra = df[df.Team.isin(non_stratifiable)]
  return extra, data_to_strat

def merge_non_stratified_with_train(x_train, y_train, extra):
    x_train = pd.concat([x_train,extra[['Problem_Abstract']]], axis=0)
    y_train = pd.concat([y_train,extra[['Team']]], axis=0)
    return x_train, y_train

In [8]:
df = load_file(_file)

In [4]:
data = load_file(os.path.join(data_folder, _file))
data = data.fillna('')
data = select_cutoff(data, 1)
extra, data_to_strat = get_non_stratifiable(data, 2)
x_train, x_test, y_train, y_test = train_test_split(data_to_strat[['Problem_Abstract']], 
                                                    data_to_strat[['Team']], 
                                                    test_size=0.1, 
                                                    stratify=data_to_strat[['Team']])
x_train, y_train = merge_non_stratified_with_train(x_train, y_train, extra)

In [5]:
import numpy as np
emb_path = 'att_hackathon/data/embeddings_mpnet.npy'
embedding = np.load(emb_path)

x_train = embedding[y_train.index]
x_test = embedding[y_test.index]

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from catboost import CatBoostClassifier
#import xgboost as xgb
#classifier = xgb.XGBClassifier(n_jobs=8, n_estimators=100, verbosity=3)
classifier = CatBoostClassifier(task_type='GPU', iterations=500)
classifier.fit(x_train, y_train)

In [ ]:
y_predicted = classifier.predict(x_test)
from sklearn.metrics import classification_report
print(classification_report(y_predicted, y_test))

In [8]:
### EXPERIMENTING WITH TFIDF
vect = TfidfVectorizer(strip_accents='unicode',
                      stop_words='english',
                      ngram_range=(1,5))
vect.fit(x_train.fillna('').Problem_Abstract)
x_train = vect.transform(x_train.fillna('').Problem_Abstract)
x_test = vect.transform(x_test.fillna('').Problem_Abstract)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

TfidfVectorizer(ngram_range=(1, 5), stop_words='english',
                strip_accents='unicode')